In [2]:
import os

import sys
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN\\src")
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN")
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN\\venv")

path = os.getcwd()

curr_path = path.replace('\\', '/')
head_place = curr_path.find('GIEBN')
main_path = curr_path[:head_place]+'GIEBN'
main_path

'c:/Users/User/PycharmProjects/GIEBN'

In [3]:
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, GradientBoostingRegressor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline, make_union
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor


from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel, f_regression

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, RFE

from auxiliary.auxiliary import construct_by_kmeans

from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.models import BayesianNetwork
import yodo.yodo
import matplotlib.pyplot as plt
from pgmpy.estimators import PC, BicScore, K2Score
from pgmpy.inference import VariableElimination
from bamt.networks.discrete_bn import DiscreteBN
from bamt_inherited.networks.discrete import DiscreteBNGI
import shap
from sklearn import metrics as m
from tqdm import tqdm
from itertools import product
import optuna
from copy import copy

In [4]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [5]:
%run ../common_code.ipynb

In [24]:
#https://www.kaggle.com/datasets/camnugent/california-housing-prices
data = pd.read_csv("../../datasets/house_16H.csv")

n = data.shape[0]
target = "price"
features = ['P1', 'P5p1', 'P6p2', 'P11p4', 'P14p9', 'P15p1', 'P15p3', 'P16p2',
       'P18p2', 'P27p4', 'H2p2', 'H8p2', 'H10p1', 'H13p1', 'H18pA', 'H40p4']
data = data[features+[target]]
data['P1']=data['P1'].apply(lambda x: (x-data['P1'].min())/(data['P1'].max()-data['P1'].min()))
data[target]=data[target].apply(lambda x: (x-data[target].min())/(data[target].max()-data[target].min()))
n_features = len(features)
data.head(10)

,P1,P5p1,P6p2,P11p4,P14p9,P15p1,P15p3,P16p2,P18p2,P27p4,H2p2,H8p2,H10p1,H13p1,H18pA,H40p4,price
0,0.002118,0.460869,0.049252,0.226470,0.149827,0.752837,0.010057,0.579729,0.003251,0.075912,0.625318,0.036613,0.991377,0.260116,0.052246,0.774059,0.261199
1,0.000211,0.470968,0.002581,0.137419,0.096341,0.862581,0.000000,0.695142,0.005025,0.043551,0.064263,0.003350,0.994975,0.285266,0.060606,0.142857,0.081000
2,0.000647,0.485341,0.000211,0.189412,0.135656,0.856992,0.000000,0.683584,0.004143,0.027965,0.065796,0.000000,0.997411,0.315433,0.065116,0.687500,0.057400
3,0.000064,0.498929,0.000000,0.100642,0.085470,0.907923,0.000000,0.780488,0.006098,0.018293,0.057471,0.000000,1.000000,0.149425,0.139535,1.000000,0.057000
4,0.000042,0.474193,0.680645,0.225806,0.128834,0.896774,0.000000,0.756302,0.008403,0.016807,0.077519,0.672269,0.991597,0.147287,0.000000,0.000000,0.048200
5,0.000063,0.475054,0.086768,0.130152,0.095041,0.924078,0.000000,0.792683,0.000000,0.006098,0.088889,0.054878,0.993902,0.272222,0.294118,0.500000,0.029998
6,0.000098,0.506224,0.000000,0.121715,0.109244,0.865837,0.000000,0.724138,0.011494,0.072797,0.100000,0.000000,1.000000,0.189655,0.146342,0.000000,0.066400
7,0.010341,0.468334,0.291213,0.169686,0.123913,0.795254,0.009719,0.614848,0.002398,0.062126,0.083928,0.263982,0.987077,0.381526,0.024669,0.248780,0.132600
8,0.000402,0.467955,0.004408,0.180400,0.135755,0.828416,0.012886,0.674938,0.002481,0.047974,0.197211,0.004136,0.995037,0.235060,0.042857,0.571429,0.275999
9,0.000613,0.509800,0.007350,0.051893,0.035438,0.950779,0.003118,0.885986,0.001584,0.037213,0.026965,0.009501,0.777514,0.190293,0.245370,0.000000,0.281199


In [25]:
data.describe()

,P1,P5p1,P6p2,P11p4,P14p9,P15p1,P15p3,P16p2,P18p2,P27p4,H2p2,H8p2,H10p1,H13p1,H18pA,H40p4,price
count,22784.000000,22784.000000,22784.000000,22784.000000,22784.000000,22784.000000,22784.000000,22784.000000,22784.000000,22784.000000,22784.000000,22784.000000,22784.000000,22784.000000,22784.000000,22784.000000,22784.000000
mean,0.001066,0.482297,0.063982,0.163900,0.121538,0.851076,0.020487,0.716131,0.003589,0.033263,0.110527,0.057437,0.967045,0.304993,0.107377,0.491626,0.100149
std,0.008996,0.031876,0.150921,0.080461,0.056687,0.079455,0.057666,0.087264,0.005100,0.029567,0.105925,0.139811,0.099953,0.133998,0.090065,0.331655,0.105687
min,0.000000,0.125000,0.000000,0.000000,0.000000,0.054156,0.000000,0.233702,0.000000,0.000000,0.000000,0.000000,0.003257,0.000000,0.000000,0.000000,0.000000
25%,0.000058,0.464647,0.000000,0.112069,0.081236,0.819686,0.000000,0.662283,0.000000,0.016260,0.049303,0.000000,0.981410,0.221673,0.052770,0.243243,0.042000
50%,0.000183,0.480396,0.003413,0.155485,0.117127,0.863645,0.000000,0.714286,0.002591,0.027523,0.081179,0.002538,0.994100,0.299800,0.086957,0.500000,0.066400
75%,0.000617,0.496040,0.033376,0.203596,0.156757,0.899657,0.019196,0.771039,0.005007,0.042827,0.131430,0.029928,1.000000,0.375000,0.137931,0.750000,0.112200
max,1.000000,0.923077,1.000000,0.917255,0.511872,1.000000,0.943325,1.000000,0.125000,0.705736,0.975177,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [26]:
data_corr = add_extra_features(data, features, extra_size=0.5, mode='random')

In [27]:
data_dict_corr = {}

data_tv, data_dict_corr['test'] = train_test_split(data_corr, test_size=0.15, random_state=42)

data_dict_corr['train'], data_dict_corr['val'] = train_test_split(data_tv, test_size=0.15, random_state=42)

features_corr = list(data_corr.columns.difference([target]))

In [28]:
features_corr

['H10p1',
 'H13p1',
 'H18pA',
 'H2p2',
 'H40p4',
 'H8p2',
 'P1',
 'P11p4',
 'P14p9',
 'P15p1',
 'P15p3',
 'P16p2',
 'P18p2',
 'P27p4',
 'P5p1',
 'P6p2',
 'extra_1',
 'extra_10',
 'extra_11',
 'extra_12',
 'extra_13',
 'extra_14',
 'extra_15',
 'extra_16',
 'extra_2',
 'extra_3',
 'extra_4',
 'extra_5',
 'extra_6',
 'extra_7',
 'extra_8',
 'extra_9']

In [29]:
def objective(trial, features, target, data_dict):
    n_estimators= trial.suggest_int("n_estimators", low=params_net['n_estimators'][0], high=params_net['n_estimators'][1], step=1)
    max_depth= trial.suggest_int("max_depth", low=params_net['max_depth'][0], high=params_net['max_depth'][1], step=1)
    learning_rate=trial.suggest_float("learning_rate", low=params_net['learning_rate'][0], high=params_net['learning_rate'][1],log=True)
    
    regr = GradientBoostingRegressor(random_state=42, n_estimators=n_estimators,
                                     max_depth=max_depth, learning_rate=learning_rate)
    regr.fit(data_dict['train'][features], data_dict['train'][target])
    return score_regr(regr, data_dict['val'][features], data_dict['val'][target])

params_net = {
    'n_estimators': [10, 100],
    'max_depth': [3, 10],
    'learning_rate':[1e-5, 1]
}


# Initial model for error calculation

In [30]:
best_base_params = optunize_model(objective, features_corr, target, data_dict_corr)

  0%|          | 0/30 [00:00<?, ?it/s]

[W 2025-05-04 23:49:11,254] Trial 1 failed with parameters: {'n_estimators': 64, 'max_depth': 4, 'learning_rate': 6.025215736203858e-05} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\User\PycharmProjects\GIEBN\venv\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_22784\2871523904.py", line 12, in <lambda>
    study.optimize(lambda trial: objective(trial, features, target, data_dict), n_trials=n_trials, show_progress_bar=True) #objective is implemented outside
  File "C:\Users\User\AppData\Local\Temp\ipykernel_22784\3082831671.py", line 8, in objective
    regr.fit(data_dict['train'][features], data_dict['train'][target])
  File "c:\Users\User\PycharmProjects\GIEBN\venv\lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\User\PycharmProjects\GIEBN\venv\

KeyboardInterrupt: 

In [31]:
best_base_params

{'n_estimators': 65, 'max_depth': 3, 'learning_rate': 0.3065986129848709}

In [40]:
model_corr = GradientBoostingRegressor(random_state=42, **best_base_params)
#model = LogisticRegression()
model_corr.fit(data_dict_corr['train'][features_corr], data_dict_corr['train'][target])

data_errors_corr = measure_error(data_dict_corr['val'], model_corr, features_corr, target, error_col_name='ape_error', task='regr')

In [41]:
(data_errors_corr['ape_error']>100000).sum()

6

In [42]:
data_errors_corr = data_errors_corr[data_errors_corr['ape_error']<=100000].reset_index(drop=True)

In [44]:
n_features_corr = len(features_corr)
disc_data_corr, pipeline_data_corr = discretize(data_errors_corr, 
                                      features_corr+["ape_error"], icat=None, icont=list(range(n_features_corr+1)))

categories_corr = extract_categories(disc_data_corr)
encoder = OneHotEncoder(sparse=False)
disc_data_encoded_corr = pd.DataFrame(encoder.fit_transform(X=disc_data_corr), columns=categories_corr, dtype='uint8')
disc_data_encoded_corr.head(1)

,P1_0,P1_1,P1_2,P5p1_0,P5p1_1,P5p1_2,P6p2_1,P6p2_2,P11p4_0,P11p4_1,...,extra_14_2,extra_15_0,extra_15_1,extra_15_2,extra_16_0,extra_16_1,extra_16_2,ape_error_0,ape_error_1,ape_error_2
0,0,1,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,0,1,0


In [45]:
disc_data_corr['ape_error'].value_counts()

1    1159
2     870
0     870
Name: ape_error, dtype: int64

In [ ]:
feat_to_grad_corr = {}
for feat in features_corr+['ape_error']:
    feat_to_grad_corr[feat]=[grad for grad in disc_data_encoded_corr.columns if feat in grad]


metric_dict = {
    'mape': lambda x, y: m.mean_absolute_percentage_error(x,y)*100,
    'r2_score': lambda x, y: m.r2_score(x, y)*100
}

gbn_fs = DiscreteBNGI('')

gbn_training_pipeline(gbn_fs, disc_data_encoded_corr, 'ape_error', categories_corr, score=K2Score)

In [55]:
gbn_fs.get_info()

,name,node_type,data_type,parents,parents_types
0,longitude_0,Discrete,disc,[],[]
1,longitude_1,Discrete,disc,[],[]
2,longitude_2,Discrete,disc,[],[]
3,latitude_0,Discrete,disc,[],[]
4,latitude_1,Discrete,disc,[],[]
...,...,...,...,...,...
94,extra_24_1,Discrete,disc,[],[]
95,extra_24_2,Discrete,disc,[],[]
96,ape_error_0,Discrete,disc,[],[]
97,ape_error_1,Discrete,disc,[],[]


In [54]:
_df=gbn_fs.get_info()
_df[_df['name'].apply(lambda x: 'ape_error' in x.name)]

,name,node_type,data_type,parents,parents_types
96,ape_error_0,Discrete,disc,[],[]
97,ape_error_1,Discrete,disc,[],[]
98,ape_error_2,Discrete,disc,[],[]


# Univariate Feature selector

In [57]:
univ_sel = SelectKBest(f_regression, k=len(features_corr))
univ_sel.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
df_val_corr_univ = pd.DataFrame({'Feature': univ_sel.feature_names_in_, 'Metric value': univ_sel.pvalues_})

# RFE on decision tree

In [58]:
rfe_sel = RFE(estimator=DecisionTreeRegressor(random_state=42, max_depth=10,), n_features_to_select=1)
#rfe_sel = RFE(estimator=RandomForestClassifier(random_state=42, max_depth=10, n_estimators=50), n_features_to_select=1, )
rfe_sel.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
df_val_corr_rfe = pd.DataFrame({'Feature': rfe_sel.feature_names_in_, 'Metric value': rfe_sel.ranking_})

# Lasso from Logistic Regression with L1 penalty

In [59]:
lsvc = Lasso(alpha=5e3, random_state=42)
lsvc.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
lasso_selector = SelectFromModel(lsvc, prefit=True)

In [60]:
lasso_support = [x for i, x in enumerate(features_corr) if lasso_selector.get_support()[i]]
lasso_support

['extra_1',
 'extra_10',
 'extra_17',
 'extra_19',
 'extra_20',
 'extra_21',
 'extra_22',
 'extra_3',
 'extra_5',
 'extra_8',
 'extra_9',
 'households',
 'housing_median_age',
 'latitude',
 'longitude',
 'median_income',
 'population',
 'total_bedrooms',
 'total_rooms']

## GBN selector

In [51]:
df_val_corr_bn=error_based_importance_features_bn(gbn_fs, 'ape_error', features_corr, feat_to_grad_corr).fillna(0)

  0%|          | 0/32 [00:00<?, ?it/s]

100%|██████████| 32/32 [00:06<00:00,  5.13it/s]


## GBN selector run

In [24]:
gbn_res = elimination_experiment(model_corr, data_dict_corr, metric_dict, df_val_corr_bn, direction='asc', task='regr')

100%|██████████| 32/32 [25:23<00:00, 47.61s/it]


In [25]:
gbn_res.head()

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,32,-,18.442,81.637,0,0
1,31,extra_9,18.454,81.52,0.064,-0.143
2,30,extra_7,18.469,81.447,0.144,-0.233
3,29,extra_24,18.533,81.509,0.496,-0.157
4,28,total_bedrooms,18.417,81.068,-0.135,-0.697


In [26]:
get_best_record(gbn_res, 'mape', mode='min')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
4,28,total_bedrooms,18.417,81.068,-0.135,-0.697


In [27]:
get_best_record(gbn_res, 'r2_score')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,32,-,18.442,81.637,0,0


In [61]:
df_corr_excluded_asc_univ=elimination_experiment(model_corr, data_dict_corr, metric_dict,
                       df_val_corr_univ, 'asc', task='regr')
df_corr_excluded_asc_univ.head()

100%|██████████| 32/32 [43:30<00:00, 81.59s/it]  


,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,32,-,19.022,80.722,0,0
1,31,median_income,20.868,78.367,9.706,-2.917
2,30,extra_23,20.928,78.758,10.02,-2.433
3,29,extra_18,21.316,78.221,12.059,-3.098
4,28,extra_6,21.099,78.092,10.921,-3.259


In [62]:
get_best_record(df_corr_excluded_asc_univ, 'mape', mode='min')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,32,-,19.022,80.722,0,0


In [63]:
get_best_record(df_corr_excluded_asc_univ, 'r2_score')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
6,26,extra_14,19.241,81.596,1.153,1.082


In [64]:
df_corr_excluded_rfe=elimination_experiment(model_corr, data_dict_corr, metric_dict,
                       df_val_corr_rfe, 'desc', task='regr')
df_corr_excluded_rfe.head()

100%|██████████| 32/32 [27:44<00:00, 52.02s/it]  


,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,32,-,18.999,80.767,0,0
1,31,extra_19,18.875,80.938,-0.654,0.212
2,30,extra_7,18.893,81.275,-0.556,0.629
3,29,extra_17,18.815,80.995,-0.967,0.283
4,28,extra_8,18.934,80.781,-0.342,0.017


In [65]:
get_best_record(df_corr_excluded_rfe, 'mape', mode='min')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
29,3,housing_median_age,18.185,81.97,-4.286,1.49


In [66]:
get_best_record(df_corr_excluded_rfe, 'r2_score')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
29,3,housing_median_age,18.185,81.97,-4.286,1.49


In [67]:
single_run(model_corr, data_dict_corr, lasso_support, target, metric_dict, task='regr')

[18.586, 81.392]

In [ ]:
#logging mape (probably woth to be negative mape)
0.5 random, base 18.467, gbn 17.877, univ 18.467, rfe 18.103 , lasso: 17.89
0.75 random, base 19.137, gbn 18.346, univ 19.137, rfe 18.239 , lasso: 18.338
0.5 noisy, base 18.337, gbn 17.927, univ 18.337, rfe  18.109, lasso: 18.221
0.75 noisy, base 18.442, gbn 17.960, univ 18.442, rfe  18.185, lasso: 18.586
0.5 prod, base 17.865, gbn 17.865, univ 17.865, rfe 17.802, lasso: 17.762
0.75 prod, base 18.442, gbn 18.417, univ 18.382, rfe 18.029, lasso: 18.585


In [ ]:
# logging r2_score
0.5 random, base 81.675, gbn 82.667, univ 81.675, rfe  82.27, lasso: 82.616 
0.75 random, base 80.482, gbn 81.832, univ 80.619, rfe  82.188, lasso: 81.806
0.5 noisy, base 81.813, gbn 82.479, univ 81.863, rfe  82.047, lasso: 81.88
0.75 noisy, base 81.637, gbn 82.039, univ 81.596, rfe  81.97, lasso: 81.392
0.5 prod, base 81.97, gbn 81.97, univ 82.176, rfe 82.438, lasso: 82.363
0.75 prod, base 81.637, gbn 81.637, univ 81.704, rfe 82.364, lasso: 80.941


In [ ]:
# #logging mape
# 0.5 random, base 17.877, gbn 80.075, univ 79.953, rfe 79.953, lasso: 79.944
# 0.75 random, base 80.119, gbn 80.55	, univ 80.119, rfe 80.119, lasso: 80.242

# 0.5 noisy base 80.42, gbn 80.58	, univ 80.42, rfe 80.578, lasso: 80.515
# 0.75 noisy base 80.264, gbn 80.311, univ 80.264, rfe 80.264, lasso: 80.633

# 0.5 prod base 80.007, gbn 80.203	, univ 80.022, rfe 80.448, lasso: 80.422
# 0.75 prod base 80.36, gbn 80.422	, univ 80.549, rfe 80.542, lasso: 80.316
